In [1]:
from notebooks.presets import set_cwd
set_cwd()
import numpy as np
import matplotlib.pyplot as plt

from src.embeddings.magnitude_spectrum import embedding_vector
from src.embeddings.seq2img_embeddings import s2i_vector
from src.embeddings.shape_based import paa, sax
from src.embeddings.simple_statistics import simple_statistics_vector
from src.data_generation.simple_data_generation import shift_time_series, shrunk_time_series

In [2]:
data_path = "data/random_sequences/random_sequences.npy"
data = np.load(data_path)

In [3]:
i = 0
n = 5
random_shifts = (np.random.random(n) - 0.5) * len(data[i]) / 4
random_shifts = random_shifts.astype(int)

def apply_embedding_function_shifts(data: np.ndarray, embedding_function: callable):
    all_embedding_vectors = []

    for i in range(data.shape[0]):
        embedding_vectors = [embedding_function(data[i])]

        for shift in random_shifts:
            shifted = shift_time_series(data[i], shift)
            embedding_vectors.append(embedding_function(shifted))

        all_embedding_vectors.append(embedding_vectors)
    return np.array(all_embedding_vectors)



In [4]:
n = 5
random_shrunks = np.random.random(n) / 4

def apply_embedding_function_shrunk(data: np.ndarray, embedding_function: callable):
    all_embedding_vectors = []

    for i in range(data.shape[0]):
        embedding_vectors = [embedding_function(data[i])]

        for shrunk in random_shrunks:
            shifted = shrunk_time_series(data[i], shrunk)
            embedding_vectors.append(embedding_function(shifted))

        all_embedding_vectors.append(embedding_vectors)
    return np.array(all_embedding_vectors)


In [5]:
embeddings = apply_embedding_function_shrunk(data, simple_statistics_vector)
embeddings.shape

(100, 6, 9)

In [18]:
stds_local = np.std(embeddings, axis=1)
stds_local.shape # 100 examples with 9 feature stds

(100, 9)

In [20]:
stds_global = np.std(embeddings, axis=(0,1))
stds_global.shape # 9 feature level stds

(9,)

In [23]:
print(stds_global)
print(np.mean(stds_local, axis=0))

[1.14751091 1.21791245 0.72487538 2.57942846 4.30063665 2.55226449
 2.40875085 1.32456645 0.68934431]
[0.00124495 0.01290489 0.02200254 0.09956409 0.05034275 0.02703494
 0.02765154 0.02987396 0.01947382]
